In [ ]:
# ===================================================== Download Stock Price and Fundamental Data ===================================================== #
import os
import time
import pandas as pd
import numpy as np
import yfinance as yf
import nasdaqdatalink as dtalink
import quandl
import pandas_datareader.data as pdr # to access FRED
from pandas_datareader.famafrench import  get_available_datasets
from functools import reduce

# import technical indicators
from talib_technical_indicators import  get_tech_indicators,  get_pattern_recognitions

# import the talib library to compute technical indicators
import talib

# enter the FRED API key
fred_api_key = os.getenv("./API/.fred_apikey")

# enter the NASDAQ Data Link API key
# dtalink.ApiConfig.verify_ssl = False
dtalink.read_key("./API/.dtalink_apikey_mc")

# Define a function to download stock, bond, inflation data
def get_stock_data(ticker: str, start_date: str, 
                                end_date: str, 
                                out_dir = 'e:/Copy/SCRIPTS/Forecast_Stocks/Data/'):
    """
        INPUT
            ticker: a Yahoo! Finance stock ticker
            start_date, end_date: start and end dates of data
            out_dir: a path to the directory where data will be stored
        OUTPUT
            a dataframe of all variables to be downloaded
    """
    stock = yf.Ticker(ticker)
    stock_df = stock.history(start=start_date, end=end_date).tz_localize(None)
    stock_df['return'] = (stock_df.Close + stock_df.Dividends) / ( stock_df.Close.shift(1) + stock_df.Dividends.shift(1) ) - 1.
    stock_df['log_return'] = np.log(stock_df['return'] + 1.)
    stock_df['price'] = stock_df.Close
    stock_df['Dividends_ffill'] = stock_df.Dividends.where(stock_df.Dividends > 0, np.nan).fillna(method = 'ffill', axis = 0)
    # stock_df['dp'] = stock_df.Dividends_ffill / stock_df.Close
    stock_df.drop(columns = ['Dividends_ffill'], inplace = True)

    # do the log-transform of volumes
    stock_df['log_volume'] = np.log(stock_df['Volume'])

    # get risk-free rate from Fama & French's data library
    ff_factors_daily_df = pdr.DataReader('F-F_Research_Data_Factors_daily', 'famafrench',  start = start_date, end = end_date, api_key = fred_api_key)
    stock_df = pd.merge(stock_df, ff_factors_daily_df[0].RF / 100, how = 'inner', left_index=True, right_index=True).drop_duplicates(keep='first')

    # get 3-month T-Bill rate from FRED
    DGS3MON_df = pdr.DataReader('DGS3MO', 'fred', start = start_date, end = end_date, api_key = fred_api_key)
    DGS3MON_df.rename(columns = {'DGS3MO': 'RF'}, inplace = True)
    stock_df = pd.merge(stock_df, DGS3MON_df.RF / 100, how = 'inner', left_index=True, right_index=True).drop_duplicates(keep='first')

    # get the ICE BofA US Corporate Index Total Return Index
    corp_bond_df = pdr.DataReader('BAMLCC0A0CMTRIV', 'fred', start = start_date, end = end_date, api_key = fred_api_key)
    corp_bond_df.rename(columns = {'BAMLCC0A0CMTRIV': 'bond_ret'}, inplace = True)
    stock_df = pd.merge(stock_df, corp_bond_df.bond_ret / 100, how = 'inner', left_index=True, right_index=True).drop_duplicates(keep='first')

    # get U.S. Treasury yields from FRED
    ten_year_treasury = pdr.DataReader('DGS10', 'fred', start = start_date, end = end_date, api_key = fred_api_key)
    three_month_treasury = pdr.DataReader('DTB3', 'fred', start = start_date, end = end_date, api_key = fred_api_key)
    treasury_df = pd.merge(ten_year_treasury, three_month_treasury, left_index=True, right_index=True, how = 'inner')
    treasury_df['term_spread'] = treasury_df['DGS10'] - treasury_df['DTB3']
    stock_df = pd.merge(stock_df, treasury_df.term_spread, how = 'inner', left_index=True, right_index=True).drop_duplicates(keep='first')

    # get S&P 500 EPS from NASDAQ Data Link
    spy_eps = dtalink.get('MULTPL/SP500_EARNINGS_MONTH', start_date = start_date, end_date = end_date)
    spy_eps.rename(columns = {'Value': 'EPS'}, inplace = True)
    stock_df = pd.merge(stock_df, spy_eps, how = 'left', left_index=True, right_index=True).drop_duplicates(keep='first')
    stock_df.EPS = stock_df.EPS.fillna(method = 'ffill', axis = 0)

    # get corporate bond spread from FRED
    credit_spread_df = pdr.DataReader('BAA10YM', 'fred', start = start_date, end = end_date, api_key = fred_api_key)
    credit_spread_df.rename(columns = {'BAA10YM': 'credit_spread'}, inplace = True)
    stock_df = pd.merge(stock_df, credit_spread_df, how = 'left', left_index=True, right_index=True).drop_duplicates(keep='first')
    stock_df.credit_spread = stock_df.credit_spread.fillna(method = 'ffill', axis = 0)

    # get FRED 10-year inflation expectation
    inflation_df = pdr.DataReader('T10YIE', 'fred', start = start_date, end = end_date, api_key = fred_api_key)
    stock_df = pd.merge(stock_df, inflation_df, how = 'inner', left_index=True, right_index=True).drop_duplicates(keep='first').rename_axis('date').reset_index()

    # save data to a CSV file
    stock_df.to_csv(os.path.join(out_dir, '%s.csv' % ticker), index = False, header = True)
    return stock_df

if __name__ == '__main__':
    # Start timing
    start_time = time.time()  
    
    # ##### Download data
    # ticker = 'SPY'
    ticker = 'BTC-USD'
    start_date = '2010-01-01'
    end_date = '2022-09-01'

    out_dir = 'e:/Dropbox/Codes/ASRock-X99/Forecast_Stocks/Data/BTC-USD/'

    stock_df = get_stock_data(ticker, start_date, end_date, out_dir=out_dir)
    stock_df = stock_df.drop_duplicates(subset = ['date'], keep='first').reset_index(drop = True)
    stock_df.Volume = stock_df.Volume.astype(np.float64)
    
    ##### Read the downloaded data into a dataframe
    stock_df = pd.read_csv(os.path.join(out_dir, '%s.csv' % ticker), encoding = 'utf-8', header = 0, skiprows = 0, skipinitialspace = True, parse_dates=['date'])
    display(stock_df.head())
    sp500_pe_df = pd.read_csv(os.path.join(out_dir, 'SP500_PE.csv'), encoding = 'utf-8', header = 0, skiprows = 0, skipinitialspace = True, parse_dates=['date'])
    
    # merge all dataframes
    stock_df = pd.merge(stock_df, sp500_pe_df, how = 'left', on = 'date').drop_duplicates(keep='first')
    stock_df['sp500_pe'] = stock_df['sp500_pe'].fillna(method = 'bfill', axis = 0)
    display(stock_df.head(90))
    

    # Compute the technical indicators
    columns_to_drop = stock_df.columns[1:].tolist()
    stock_dfs = []
    stock_dfs.append(stock_df.loc[:, ~stock_df.columns.isin(['Open',	'High',	'Low',	'Close',	'Volume',	'Dividends',	'Stock Splits'])])

    timeperiods = [14, 24, 34, 54, 104]
    for timeperiod in timeperiods:
        df = get_tech_indicators(stock_df, ticker, timeperiod, out_dir=out_dir)
        df.drop(columns = columns_to_drop, axis = 1, inplace = True)
        stock_dfs.append(df)

    # Merge all the dataframes and remove duplicate columns
    df_final = reduce(lambda left, right: pd.merge( left, right, on='date', suffixes=('', '_DROP') ).filter(regex='^(?!.*_DROP)'), stock_dfs) 
    df_final.insert(loc = 1, column = 'direction', value = (stock_df['return'] > 0).astype('int'))

    df_final.dropna(inplace = True)
    df_final = df_final.drop_duplicates(keep='first').reset_index(drop = True)
    df_final = df_final.loc[:, ~df_final.T.duplicated(keep='first')] # remove duplicate columns

    df_final.to_csv(os.path.join(out_dir, '%s_all_vars.csv' % ticker), index = False, header = True)
    display( df_final.head() )

    # Compute the pattern recognition indicators
    stock_df = get_pattern_recognitions(stock_df, ticker, out_dir=out_dir)
    df_final = pd.merge(df_final.set_index('date'), stock_df.set_index('date').loc[:, 'CDL2CROWS':], how = 'left', left_index = True, right_index = True)
    df_final.dropna(inplace = True)
    df_final.to_csv(os.path.join(out_dir, '%s_all_vars_plus_patterns.csv' % ticker), index = True, header = True)
    display( df_final.head() )

    print( 'Completed in: %s sec'%(time.time() - start_time) )


